## Test for interventions

In [2]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)

### Define a new attribute

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [3]:
from snowplow_signals import (
    Attribute,
    Event,
)

count_page_views = Attribute(
    name="count_page_views",
    type="int32",
    events=[
        Event(name="page_view")
    ],
    aggregation="counter",
)

last_page_title = Attribute(
    name="last_page_title",
    type="string",
    events=[
        Event(name="page_view")
    ],
    aggregation="last",
    property="unstruct_event_com_hello_fresh_explib_decide_2:experiment_key",
)

### Wrapping the attribute in a view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [4]:
from snowplow_signals import View, LinkEntity

view = View(
    name="page_view_attributes",
    version=1,
    entity=LinkEntity(name="domain_userid"),
    attributes=[
        count_page_views,
    ],
    owner="user@company.com",
)

### Testing the view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [5]:
data = sp_signals.test(
    view=view,
    app_ids=["website"],
)
data

,domain_userid,count_page_views
0,5de568e2-2b3c-41e1-a207-b2093896d274,1
1,8d572123-c176-43b7-90a2-3401e868faa8,1
2,d0709483-40a7-41cd-b21b-b93dc14fb9df,1
3,4bf3b7f2-049d-47be-9edc-aa6183a2b769,0
4,a5ba78d3-2d34-4eae-8715-a4d287a7d976,1
5,fc46fd44-adb5-4ece-98ba-c2efa7e16d3d,2
6,9cc599a1-2355-49e4-ad47-836e68c2ca15,1
7,a6a7f83a-baea-4753-9b17-e18a9d7e7df1,1
8,2315b033-e166-4094-ad38-43c73f15ba7d,2
9,02f19737-7b38-40d5-9e16-392417788516,2


In [5]:
from snowplow_signals import (
    RuleIntervention,
    InterventionSetAttributeContext,
    InterventionCriterion,
)

intervention = RuleIntervention(
    name="page_view_count",
    description="Resets the number of add_to_cart events when it becomes more than three.",
    method="set_attribute",
    context=InterventionSetAttributeContext(
        attribute="page_view_attributes:count_page_views",
        value=3,
        clear_history=True,
    ),
    criteria=InterventionCriterion(
        attribute="page_view_attributes:count_page_views",
        operator=">",
        value=3,
    ),
)

### Define new intervention

TODO: Shouldn't this be a part of the `apply()` function below?

In [10]:
sp_signals.interventions.create(intervention)

RuleInterventionOutput(name='page_view_count', version=1, method='set_attribute', target_agents=None, script_uri=None, context=SetAttributeContext(field_attributes=None, attribute='page_view_attributes:count_page_views', value=3, path=None, clear_history=True), description='Resets the number of add_to_cart events when it becomes more than three.', tags=None, owner=None, criteria=SignalsApiModelsInterventionCriterionCriterion(attribute='page_view_attributes:count_page_views', operator='>', value=3))

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [11]:
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

1 objects applied


### Retrieving interventions

TODO: We need a better flow for testing interventions by users.

In [9]:
import requests
import sseclient
import os
import json

API_URL = f"{os.environ["SP_API_URL"]}/api/v1/interventions"

headers = {
    "Accept": "text/event-stream",
}

params = {
    "domain_userid": "26e2769d-ee31-43d5-970d-bfa598c4f88e"
}

with requests.get(API_URL, headers=headers, params=params, stream=True) as response:
    if response.status_code != 200:
        print(f"Error: {response.status_code} {response.reason}")
    else:
        client = sseclient.SSEClient(response)
        for event in client.events():
            print(json.dumps(json.loads(event.data), indent=2))

{
  "method": "set_attribute",
  "target_agents": null,
  "script_uri": null,
  "context": {
    "$attributes": {
      "JM_test_view_1_stream:pv_count": 6,
      "JM_test_view_1_stream:recent_view": "2025-05-26T14:33:45.405Z",
      "demo_landing_page_1747756785_stream:latest_app_id": "website",
      "demo_landing_page_1747756785_stream:latest_device_class": "Desktop",
      "demo_landing_page_1747756785_stream:num_apps_l30d": [
        "website"
      ],
      "demo_landing_page_1747756785_stream:num_apps_l7d": [
        "website"
      ],
      "demo_landing_page_1747756785_stream:num_page_pings_l30d": 3,
      "demo_landing_page_1747756785_stream:num_page_pings_l7d": 3,
      "demo_landing_page_1747756785_stream:num_page_views_l30d": 6,
      "demo_landing_page_1747756785_stream:num_page_views_l7d": 6,
      "demo_landing_page_1747756785_stream:num_sessions_l30d": [
        "1a89bf27-ac1f-4e7f-af64-3efca196c0e8"
      ],
      "demo_landing_page_1747756785_stream:num_sessions_l7d"

KeyboardInterrupt: 